In [2]:
import pandas as pd

In [3]:
data_path = "/data_nas/cehou/LLM_safety/PlacePulse2.0/metadata/final_data.csv"
data = pd.read_csv(data_path)

In [4]:
data

,left_id,right_id,winner,left_lat,left_long,right_lat,right_long,category
0,513d2dbefdc9f03587002515,51413b2ffdc9f049260058ca,left,41.935385,-87.696485,34.047847,-118.236107,wealthy
1,513d7e23fdc9f0358700733a,513d7ac3fdc9f0358700693f,equal,40.744156,-73.935579,-33.526384,-70.591309,depressing
2,50f60c4dbeb2fed91000028d,513cb652fdc9f03587000a1e,right,45.505837,-122.583705,29.752872,-95.298844,depressing
3,513d9ff9fdc9f0358700887c,513cd544fdc9f03587001efc,left,33.747093,-84.289843,-33.930831,18.406321,depressing
4,50f60b90beb2fed910000085,513d6debfdc9f0358700521f,left,45.504360,-122.610375,39.780163,-104.955843,safety
...,...,...,...,...,...,...,...,...
1208803,513e1d28fdc9f03587009919,50f5eaaefdc9f065f0007c16,left,55.836046,37.506351,-22.882859,-43.276335,safety
1208804,513e1d32fdc9f03587009933,50f60b89beb2fed91000001f,left,55.807802,37.580988,45.562967,-122.560071,safety
1208805,514139f4fdc9f049260055b4,513d5c4afdc9f035870036b9,left,-26.178270,28.040300,-19.846557,-43.914151,safety
1208806,50e748e9d7c3df413b00144c,513cd430fdc9f03587001ebf,left,55.682180,12.605489,-33.934795,18.485469,safety


In [8]:
data[data['left_id'] == "51413b2ffdc9f049260058ca"]

,left_id,right_id,winner,left_lat,left_long,right_lat,right_long,category
121598,51413b2ffdc9f049260058ca,513e1a68fdc9f035870091d3,right,34.047847,-118.236107,55.727393,37.548291,depressing
431319,51413b2ffdc9f049260058ca,513cc044fdc9f0358700136b,left,34.047847,-118.236107,29.755390,-95.435065,depressing
519992,51413b2ffdc9f049260058ca,513e5e9bfdc9f0358700b0fd,equal,34.047847,-118.236107,48.929106,2.280332,wealthy
565279,51413b2ffdc9f049260058ca,513d7a83fdc9f0358700686b,right,34.047847,-118.236107,-33.559347,-70.558710,boring
565280,51413b2ffdc9f049260058ca,513d7a83fdc9f0358700686b,right,34.047847,-118.236107,-33.559347,-70.558710,boring
571358,51413b2ffdc9f049260058ca,513d7c3efdc9f03587006e23,right,34.047847,-118.236107,40.771691,-73.846503,beautiful
778010,51413b2ffdc9f049260058ca,513d7d33fdc9f0358700713e,right,34.047847,-118.236107,-33.429074,-70.543226,beautiful
801510,51413b2ffdc9f049260058ca,51414d58fdc9f0492600757a,left,34.047847,-118.236107,45.458436,9.210855,beautiful
830536,51413b2ffdc9f049260058ca,50f5e690fdc9f065f0006d7c,right,34.047847,-118.236107,-23.610848,-46.734408,safety
1154901,51413b2ffdc9f049260058ca,513cc01afdc9f03587001343,left,34.047847,-118.236107,41.873826,-87.735415,lively
